In [48]:
import numpy as np


## Definition of elastic tensor, \nu > 0.25

In [49]:
E = 1.
nu = 0.4

la = E * nu / ((1. + nu) * (1. - 2. * nu))
mu = E / (2. + 2. * nu)

delta = np.identity(3)

D_abef = (np.einsum(',ij,kl->ijkl', la, delta, delta) +
          np.einsum(',ik,jl->ijkl', mu, delta, delta) +
          np.einsum(',il,jk->ijkl', mu, delta, delta))

## Definition of strain and stress tensor, uniaxial compression

In [50]:
eps = np.array([[-1, 0, 0],[0, 0.4, 0],[0, 0, 0.4]])

sigma = np.einsum('ijkl,kl->ij',D_abef,eps)

print('\epsilon =' ,eps)
print('\sigma =' ,sigma)

\epsilon = [[-1.   0.   0. ]
 [ 0.   0.4  0. ]
 [ 0.   0.   0.4]]
\sigma = [[-1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -1.11022302e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.11022302e-16]]


## Kinematic constrain

\begin{align}
\varepsilon_{\mathrm{N}} &= \varepsilon_{ij} n_i n_j\\
\nonumber
\varepsilon_{\mathrm{T}_r} &= \frac{1}{2}
\left(n_i \delta_{jr} + n_j \delta_{ir} - 2 n_i n_j n_r\right) \,  \varepsilon_{ij}
\end{align}

In [ ]:
n = np.array([.250562787, -.935113132, .250562787])

eps_N = np.einsum('ij,i,j->',eps,n,n)
eps_T = 0.5 * np.einsum('i,jr,ij->r',n,delta,eps) + 0.5 * np.einsum('j,ir,ij->r',n,delta,eps)\
        - np.einsum('i,j,r,ij->r',n,n,n,eps)
print('\epsilon_N = ',eps_N)
print('\epsilon_T = ',eps_T)

## Static constrain

\begin{align}
\sigma_{\mathrm{N}} &= \sigma_{ij} n_i n_j\\
\nonumber
\sigma_{\mathrm{T}_r} &= \frac{1}{2}
\left(n_i \delta_{jr} + n_j \delta_{ir} - 2 n_i n_j n_r\right) \,  \sigma_{ij}
\end{align}

In [ ]:
sigma_N_static = np.einsum('ij,i,j->',sigma,n,n)
sigma_T_statin = 0.5 * np.einsum('i,jr,ij->r',n,delta,sigma) + 0.5 * np.einsum('j,ir,ij->r',n,delta,sigma)\
        - np.einsum('i,j,r,ij->r',n,n,n,sigma)
print('\sigma_N_static =' ,sigma_N_static)
print('\sigma_T_static =',sigma_T_statin)



## Applying microplane material law

In [ ]:
E_N = E / (1.0 - 2.0 * nu)
E_T = (E * (1.0 - 4.0 * nu)) / ((1.0 + nu) * (1.0 - 2.0 * nu))

sigma_N = E_N * eps_N
sigma_T = E_T * eps_T

print('\sigma_N_micro =' ,sigma_N)
print('\sigma_T_micro =' ,sigma_T)

## We are getting microplane stress vector going in the opposite direction of strain vector 
## What happens if we integrate the microplane stresses?

From PVW or integrating microplane free energy, same result. By: Carol, Bazant, Jirasek

\begin{equation}
\sigma_{ij} = \frac{3}{4\pi} \int_{\Omega} \sigma_{\mathrm{N}} n_i n_j d\Omega + \frac{3}{4\pi} \int_{\Omega} \frac{\sigma_{\mathrm{T}_r}}{2} (n_i \delta_{rj} + n_j \delta_{ri}) d\Omega \hspace{1cm}
\label{eq:sigma_ij}
\end{equation}

## 28 microplanes hemisphere, weights factors and normals, Bazant

In [ ]:
MPW = np.array([.0160714276, .0160714276, .0160714276, .0160714276, .0204744730,
                      .0204744730, .0204744730, .0204744730, .0204744730, .0204744730,
                      .0204744730, .0204744730, .0204744730, .0204744730, .0204744730,
                      .0204744730, .0158350505, .0158350505, .0158350505, .0158350505,
                      .0158350505, .0158350505, .0158350505, .0158350505, .0158350505,
                      .0158350505, .0158350505, .0158350505]) * 6.0

MPN = np.array([[.577350259, .577350259, .577350259],
                      [.577350259, .577350259, -.577350259],
                      [.577350259, -.577350259, .577350259],
                      [.577350259, -.577350259, -.577350259],
                      [.935113132, .250562787, .250562787],
                      [.935113132, .250562787, -.250562787],
                      [.935113132, -.250562787, .250562787],
                      [.935113132, -.250562787, -.250562787],
                      [.250562787, .935113132, .250562787],
                      [.250562787, .935113132, -.250562787],
                      [.250562787, -.935113132, .250562787],
                      [.250562787, -.935113132, -.250562787],
                      [.250562787, .250562787, .935113132],
                      [.250562787, .250562787, -.935113132],
                      [.250562787, -.250562787, .935113132],
                      [.250562787, -.250562787, -.935113132],
                      [.186156720, .694746614, .694746614],
                      [.186156720, .694746614, -.694746614],
                      [.186156720, -.694746614, .694746614],
                      [.186156720, -.694746614, -.694746614],
                      [.694746614, .186156720, .694746614],
                      [.694746614, .186156720, -.694746614],
                      [.694746614, -.186156720, .694746614],
                      [.694746614, -.186156720, -.694746614],
                      [.694746614, .694746614, .186156720],
                      [.694746614, .694746614, -.186156720],
                      [.694746614, -.694746614, .186156720],
                      [.694746614, -.694746614, -.186156720]])

## 2 order N tensor, 3rd order T tensor

In [ ]:
MPTT_ijr = 0.5 * (np.einsum('ni,jr -> nijr', MPN, delta) +
                   np.einsum('nj,ir -> njir', MPN, delta) - 2 *
                   np.einsum('ni,nj,nr -> nijr', MPN, MPN, MPN))

MPNN = np.einsum('ni,nj->nij', MPN, MPN)



## Applying kinematic constraint

In [ ]:
eps_N_Emn = np.einsum('nij,...ij->...n', MPNN, eps) 

eps_T_Emna = np.einsum('nija,...ij->...na', MPTT_ijr, eps)

print('\epsilon_N = ',eps_N_Emn[10])
print('\epsilon_T = ',eps_T_Emna[10])

## Microplane material law

In [ ]:
sigma_N_Emn = E_N * eps_N_Emn
sigma_T_Emna = E_T * eps_T_Emna

print('\sigma_N_micro =' ,sigma_N_Emn[10])
print('\sigma_T_micro =' ,sigma_T_Emna[10])

## Numerical integration of microplane stresses

In [ ]:
sigma_int = np.einsum('n,...n,na,nb->...ab', MPW, sigma_N_Emn, MPN, MPN) + \
            0.5 * (np.einsum('n,...nf,na,fb->...ab', MPW, sigma_T_Emna, MPN, delta) +
                   np.einsum('n,...nf,nb,fa->...ab', MPW, sigma_T_Emna, MPN, delta))

print('integrated \sigma = ',sigma_int)
print('original \sigma = ',sigma)

sigma_int = np.einsum('n,...n,na,nb->...ab', MPW, sigma_N_Emn, MPN, MPN) + \
            0.5 * (np.einsum('n,...nf,na,fb->...ab', MPW, sigma_T_Emna, MPN, delta) +
                   np.einsum('n,...nf,nb,fa->...ab', MPW, sigma_T_Emna, MPN, delta))

print('integrated \sigma = ',sigma_int)
print('original \sigma = ',sigma)

\begin{align}
\varepsilon_{\mathrm{N}} &= \varepsilon_{ij} n_i n_j\\
\nonumber
\varepsilon_{\mathrm{T}_r} &= \frac{1}{2}
\left(n_i \delta_{jr} + n_j \delta_{ir} - 2 n_i n_j n_r\right) \,  \varepsilon_{ij}
\end{align}

In [51]:
n = np.array([.250562787, -.935113132, .250562787])

eps_N = np.einsum('ij,i,j->',eps,n,n)
eps_T = 0.5 * np.einsum('i,jr,ij->r',n,delta,eps) + 0.5 * np.einsum('j,ir,ij->r',n,delta,eps)\
        - np.einsum('i,j,r,ij->r',n,n,n,eps)
print('\epsilon_N = ',eps_N)
print('\epsilon_T = ',eps_T)

\epsilon_N =  0.3121056017180153
\epsilon_T =  [-0.32876484 -0.08219121  0.02202307]


## Static constrain

\begin{align}
\sigma_{\mathrm{N}} &= \sigma_{ij} n_i n_j\\
\nonumber
\sigma_{\mathrm{T}_r} &= \frac{1}{2}
\left(n_i \delta_{jr} + n_j \delta_{ir} - 2 n_i n_j n_r\right) \,  \sigma_{ij}
\end{align}

In [52]:
sigma_N_static = np.einsum('ij,i,j->',sigma,n,n)
sigma_T_statin = 0.5 * np.einsum('i,jr,ij->r',n,delta,sigma) + 0.5 * np.einsum('j,ir,ij->r',n,delta,sigma)\
        - np.einsum('i,j,r,ij->r',n,n,n,sigma)
print('\sigma_N_static =' ,sigma_N_static)
print('\sigma_T_static =',sigma_T_statin)



\sigma_N_static = -0.06278171022920749
\sigma_T_static = [-0.23483203 -0.058708    0.01573076]


## Applying microplane material law

In [53]:
E_N = E / (1.0 - 2.0 * nu)
E_T = (E * (1.0 - 4.0 * nu)) / ((1.0 + nu) * (1.0 - 2.0 * nu))

sigma_N = E_N * eps_N
sigma_T = E_T * eps_T

print('\sigma_N_micro =' ,sigma_N)
print('\sigma_T_micro =' ,sigma_T)

\sigma_N_micro = 1.5605280085900768
\sigma_T_micro = [ 0.70449608  0.17612401 -0.04719228]


## We are getting microplane stress vector going in the opposite direction of strain vector 
## What happens if we integrate the microplane stresses?

From PVW or integrating microplane free energy, same result. By: Carol, Bazant, Jirasek

\begin{equation}
\sigma_{ij} = \frac{3}{4\pi} \int_{\Omega} \sigma_{\mathrm{N}} n_i n_j d\Omega + \frac{3}{4\pi} \int_{\Omega} \frac{\sigma_{\mathrm{T}_r}}{2} (n_i \delta_{rj} + n_j \delta_{ri}) d\Omega \hspace{1cm}
\label{eq:sigma_ij}
\end{equation}

## 28 microplanes hemisphere, weights factors and normals, Bazant

In [54]:
MPW = np.array([.0160714276, .0160714276, .0160714276, .0160714276, .0204744730,
                      .0204744730, .0204744730, .0204744730, .0204744730, .0204744730,
                      .0204744730, .0204744730, .0204744730, .0204744730, .0204744730,
                      .0204744730, .0158350505, .0158350505, .0158350505, .0158350505,
                      .0158350505, .0158350505, .0158350505, .0158350505, .0158350505,
                      .0158350505, .0158350505, .0158350505]) * 6.0

MPN = np.array([[.577350259, .577350259, .577350259],
                      [.577350259, .577350259, -.577350259],
                      [.577350259, -.577350259, .577350259],
                      [.577350259, -.577350259, -.577350259],
                      [.935113132, .250562787, .250562787],
                      [.935113132, .250562787, -.250562787],
                      [.935113132, -.250562787, .250562787],
                      [.935113132, -.250562787, -.250562787],
                      [.250562787, .935113132, .250562787],
                      [.250562787, .935113132, -.250562787],
                      [.250562787, -.935113132, .250562787],
                      [.250562787, -.935113132, -.250562787],
                      [.250562787, .250562787, .935113132],
                      [.250562787, .250562787, -.935113132],
                      [.250562787, -.250562787, .935113132],
                      [.250562787, -.250562787, -.935113132],
                      [.186156720, .694746614, .694746614],
                      [.186156720, .694746614, -.694746614],
                      [.186156720, -.694746614, .694746614],
                      [.186156720, -.694746614, -.694746614],
                      [.694746614, .186156720, .694746614],
                      [.694746614, .186156720, -.694746614],
                      [.694746614, -.186156720, .694746614],
                      [.694746614, -.186156720, -.694746614],
                      [.694746614, .694746614, .186156720],
                      [.694746614, .694746614, -.186156720],
                      [.694746614, -.694746614, .186156720],
                      [.694746614, -.694746614, -.186156720]])

## 2 order N tensor, 3rd order T tensor

In [55]:
MPTT_ijr = 0.5 * (np.einsum('ni,jr -> nijr', MPN, delta) +
                   np.einsum('nj,ir -> njir', MPN, delta) - 2 *
                   np.einsum('ni,nj,nr -> nijr', MPN, MPN, MPN))

MPNN = np.einsum('ni,nj->nij', MPN, MPN)



## Applying kinematic constraint

In [56]:
eps_N_Emn = np.einsum('nij,...ij->...n', MPNN, eps) 

eps_T_Emna = np.einsum('nija,...ij->...na', MPTT_ijr, eps)

print('\epsilon_N = ',eps_N_Emn[10])
print('\epsilon_T = ',eps_T_Emna[10])

\epsilon_N =  0.3121056017180154
\epsilon_T =  [-0.32876484 -0.08219121  0.02202307]


## Microplane material law

In [57]:
sigma_N_Emn = E_N * eps_N_Emn
sigma_T_Emna = E_T * eps_T_Emna

print('\sigma_N_micro =' ,sigma_N_Emn[10])
print('\sigma_T_micro =' ,sigma_T_Emna[10])

\sigma_N_micro = 1.5605280085900772
\sigma_T_micro = [ 0.70449608  0.17612401 -0.04719228]


## Numerical integration of microplane stresses

In [58]:
sigma_int = np.einsum('n,...n,na,nb->...ab', MPW, sigma_N_Emn, MPN, MPN) + \
            0.5 * (np.einsum('n,...nf,na,fb->...ab', MPW, sigma_T_Emna, MPN, delta) +
                   np.einsum('n,...nf,nb,fa->...ab', MPW, sigma_T_Emna, MPN, delta))

print('integrated \sigma = ',sigma_int)
print('original \sigma = ',sigma)

integrated \sigma =  [[-1.00000026e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.31586186e-07  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.31586186e-07]]
original \sigma =  [[-1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -1.11022302e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.11022302e-16]]
